In [2]:
import ast
import traceback

from itertools import islice
import pandas as pd
import numpy as np
# import math

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 4)
# pd.set_option('display.width', 1000)

# TODO : contact map first
1. splits logs into 3 parts, sim_write , agg_read , agg_write
2. then in each part, splits into vol and vfd logs

In [3]:


def check_empty(df,p=True):
    nan_df = df[df.isna().any(axis=1)].copy()
    if not nan_df.empty:
        if p:
            print(f"NaN rows: {nan_df}") # check any nan
        return list(nan_df.index)
    
    null_df = df[df.isnull().any(axis=1)].copy()
    if not null_df.empty:
        if p:
            print(f"NULL rows: {null_df}") # check any null
        return list(null_df.index)
    
def rec_to_df(records):
    df = pd.DataFrame.from_dict(records,orient='index')
    df.replace('/mnt/ssd/mtang11/','',regex=True, inplace=True)
    df.replace('molecular_dynamics_runs/stage0000/','',regex=True, inplace=True)
    
    # df['hash_id']= df['hash_id'].astype(str)
    # df['hash_id'] = map(lambda x: x.encode('base64','strict'), df['hash_id'])
    if 'logical_addr' in df.columns:
        df['logical_addr'] = df['logical_addr'].fillna(-1.0) #.astype(int)
    #df['logical_addr'] = df['logical_addr'].astype(int)
    
    return df

def df_to_csv(df,file_name,suffix=''):
    # df.hash_id=df.hash_id.astype('category').cat.codes
    # out_csv=file_name.replace('prov-vfd-','')
    out_csv=file_name.replace('.log',f'{suffix}.csv')
    df.to_csv(out_csv,index=False)

def df_to_parquet(df,file_name,suffix=''):
    out_parquet=file_name.replace('.log',f'{suffix}.parquet')
    try:
        df.to_parquet(out_parquet, engine='pyarrow') # compression='gzip'
        # pyarrow has error with streamlit, downgrade to streamlit==0.84.2
    except:
        traceback.print_exc()

def print_list_diff(list1, list2):
    if list1 == list2:
        print("same lists")
    else:
        print("different lists\nindex:\t[list1]\t[list2]")
        for index, (first, second) in enumerate(zip(list1, list2)):
            if first != second:
                # print(index, first, second)
                print(f"{index}:\t{first}]\t[{second}]")


def split_vfd_vol_rec(fname,mode='r'):
    vol_r_ops = ['H5VLdataset_read', 'H5VLblob_get', 'H5FD__hermes_read']
    vol_w_ops = ['H5VLdataset_write', 'H5VLblob_put', 'H5FD__hermes_write']
    other_ops = ['_get','_create', '_close', '_open',]

    vol_rec = {}
    vfd_rec = {}
    vol_idx = 0
    vfd_idx = 0
    with open(fname, mode) as f:
        for line in f:
            try:
                rec = ast.literal_eval(line)
                
                if any(op in rec['func_name'] for op in vol_r_ops):
                    if 'hermes' in rec['func_name']:
                        vfd_rec[vfd_idx] = rec
                        vfd_rec[vfd_idx]['operation'] = 'write'
                        vfd_idx+=1
                    else:
                        vol_rec[vol_idx] = rec
                        vol_rec[vol_idx]['operation'] = 'read'
                        vol_idx+=1
                elif any(op in rec['func_name'] for op in vol_w_ops):
                    if 'hermes' in rec['func_name']:
                        vfd_rec[vfd_idx] = rec
                        vfd_rec[vfd_idx]['operation'] = 'write'
                        vfd_idx+=1
                    else:
                        vol_rec[vol_idx] = rec
                        vol_rec[vol_idx]['operation'] = 'write'
                        vol_idx+=1          
                # not record other_ops
                elif any(op in rec['func_name'] for op in other_ops):
                    _suffix = rec['func_name'].split('_')[-1]
                    if 'hermes' in rec['func_name']:
                        vfd_rec[vfd_idx] = rec
                        vfd_rec[vfd_idx]['operation'] = _suffix
                        vfd_idx+=1
                    else:
                        vol_rec[vol_idx] = rec
                        vol_rec[vol_idx]['operation'] = _suffix
                        vol_idx+=1
                    
            except:
                print("Erro line:")
                print(line)
                # break
    return vol_rec, vfd_rec

def add_all_op_type(dfvol, dfvfd, vol_map={}, vfd_map={}):
    if bool(vol_map) == False:
        vol_map={'read':'data', 'write': 'data'}
        vfd_map={'H5FD_MEM_DRAW':'data', 'H5FD_MEM_LHEAP': 'lheap'}

    dfvol['op_type'] = dfvol['operation'].map(vol_map)
    dfvol['op_type'] = dfvol['op_type'].fillna('meta')
    
    dfvfd['mem_type'] = dfvfd['mem_type'].fillna('H5FD_MEM_NTYPES')
    dfvfd['op_type'] = dfvfd['mem_type'].map(vfd_map)
    dfvfd['op_type'] = dfvfd['op_type'].fillna('meta')

def add_vol_op_type(dfvol, vol_map={}):
    if bool(vol_map) == False:
        vol_map={'read':'data', 'write': 'data'}

    dfvol['op_type'] = dfvol['operation'].map(vol_map)
    dfvol['op_type'] = dfvol['op_type'].fillna('meta')


In [4]:
# read in simulation data
fsim='../save_outputs/vol-vfd/prov-vfd-sim.log'
mode='r'

vol_sim_rec, vfd_sim_rec = split_vfd_vol_rec(fsim)

vol_sdf = rec_to_df(vol_sim_rec)
vfd_sdf = rec_to_df(vfd_sim_rec)

# add category
vol_sdf['cat'] = 'sim-write'
vfd_sdf['cat'] = 'sim-write'

def vol_data_label(df):
    data_idx_list = []
    dsets = list(set(df['file_no']))
    for index, row in df.iterrows():
        _pat = row['op_type']
        _dset = row['dset_name']
        _str = f"vol-{_pat}-{_dset}"
        if _pat =='data':
            _str = f"vol-{_pat}-{_dset}-{row['io_access_idx']}"
            
        data_idx_list.append(_str)

    df['data_label'] = data_idx_list
    
def vfd_data_label_sim_write(df):
    data_idx = 0
    data_idx_list = []
    dsets = list(set(df['file_no']))
    curr_dset = dsets.pop(0)
    dset_changed = False
    for index, row in df.iterrows():
        _pat = row['op_type']
        
        if _pat =='data':
            data_idx_list.append(f"vfd-{_pat}-{curr_dset}-{data_idx}")
            data_idx+=1
        else:
            data_idx_list.append(f"vfd-{_pat}-{curr_dset}")
            
        if row['file_no'] != curr_dset:
            data_idx = 0
            curr_dset = dsets.pop(0)

    df['data_label'] = data_idx_list

def fill_dset_offset(dfvol, dfvfd, fno2dset_dict={1:'contact_map', 2:'point_cloud'}):
    dsets_no = list(fno2dset_dict.keys())
    offset_dict = {}
    for no in dsets_no:
        # find first dataset starting offset
        # vfd_sdf[vfd_sdf['data_label'] == f'vfd-data-{dset}-0'].iloc[0]['start_addr']
        
        idx = dfvfd.data_label.str.match(f'vfd-data-{no}-0').idxmax()
        START_OFFSET = dfvfd.iloc[idx]['start_addr']
        offset_dict[fno2dset_dict[no]] = START_OFFSET
    
    dfvol['offset'] = dfvol['dset_name'].map(offset_dict)
    return dfvol

def hard_code_start_addr(dfvol, dfvfd):
    # # hardcode starting address
    dfvol = fill_dset_offset(dfvol, dfvfd)
    # starting address of blob
    idx_vfd = dfvfd.op_type.str.match('data').idxmax() # obtain from first vfd access 
    BLOB_START_ADDR = dfvfd.iloc[idx_vfd]['access_size'] +  dfvfd.iloc[idx_vfd]['start_addr'] 
    idx_vol = dfvol.func_name.str.match('H5VLblob_put').idxmax()
    dfvol.loc[idx_vol, ['logical_addr']] = BLOB_START_ADDR
    # print(dfvol.iloc[idx_vol])
    
    # starting address of dset
    dset_idx = dfvol.index[dfvol['func_name'] == 'H5VLdataset_write'].tolist()
    
    for dsi in dset_idx:
        dfvol.loc[dsi, ['logical_addr']] = dfvol.iloc[dsi]['offset']
    
    return dfvol

# # bfill_cols = ['layout', 'type_size', 'n_elements', 'dimension_cnt', 'dimensions', 'dset_name', 'file_intent']

add_all_op_type(vol_sdf,vfd_sdf)
# # add data_label for simulation
vol_sdf['dset_name'] = vol_sdf['dset_name'].fillna(method='bfill')
vol_sdf['dset_name'] = vol_sdf['dset_name'].fillna(method='ffill')

vfd_data_label_sim_write(vfd_sdf)
vol_data_label(vol_sdf)

vol_sdf = hard_code_start_addr(vol_sdf,vfd_sdf)

# check_empty(vfd_sdf)

df_to_csv(vol_sdf,fsim,suffix='-vol')
df_to_csv(vfd_sdf,fsim,suffix='-vfd')

# # TODO: use parquet for faster load in later analysis
# df_to_parquet(vol_sdf,fsim,suffix='-vol') 
# df_to_parquet(vfd_sdf,fsim,suffix='-vfd')

In [5]:
fagg='../save_outputs/vol-vfd/prov-vfd-agg.log'

vol_agg_rec, vfd_agg_rec = split_vfd_vol_rec(fagg)

vol_adf = rec_to_df(vol_agg_rec)
vfd_adf = rec_to_df(vfd_agg_rec)

# add category, split read write based on filename
vol_adf['cat'] = np.where(vol_adf['file_name']== 'aggregate.h5', 'agg-write', 'agg-read')
vfd_adf['cat'] = np.where(vfd_adf['file_name']== 'aggregate.h5', 'agg-write', 'agg-read')

# add columns for mapping
vfd_adf['start_addr'] = vfd_adf['start_addr'].fillna(-1)
vfd_adf['access_size'] = vfd_adf['access_size'].fillna(0)
vfd_adf['next_addr'] = vfd_adf['start_addr'] + vfd_adf['access_size']
vfd_adf['next_addr'] = vfd_adf['next_addr'].astype(int)

In [6]:
# read in aggregation data

def vfd_op_data_label_agg_read(df, dset_offsets):

    data_label_list = []
 
    curr_dset = 1
    next_offset = dset_offsets[curr_dset]
    dset_changed = False
    
    data_idx = 1
    last_addr = -1 # vfd has no -1 address
    pattern_list = []
    
    for index, row in df.iterrows(): #islice(df.iterrows(), start_idx, None):
        if row['mem_type'] == "H5FD_MEM_DRAW":
            if row['access_size'] == 4096:
                last_addr = row['next_addr']
                data_label_list.append(f'vfd-data-{curr_dset}-{data_idx}')
                pattern_list.append('loc')
            elif row['start_addr'] == last_addr:
                data_label_list.append(f'vfd-data-{curr_dset}-{data_idx}')
                pattern_list.append('data')
                data_idx+=1
                last_addr = 0
            else:
                data_label_list.append(f'vfd-data-{curr_dset}-{0}') # first access
                pattern_list.append('data')
        else:
            data_label_list.append(f'vfd-meta-{curr_dset}')
            last_addr = row['next_addr']
            pattern_list.append('meta')
            
        if row['start_addr'] == next_offset:
            data_idx = 0
            curr_dset+=1
            if len(dset_offsets) <= curr_dset:
                next_offset = 0
            else:
                next_offset = dset_offsets[curr_dset]
            dset_changed = True
    
    data_label_list[-1] = f'vfd-data-{curr_dset}-{data_idx}' # last idx for point_cloud
    
    df['data_label'] = data_label_list
    df['op_type'] = pattern_list
    
    return df


# add mapping for data_label
vol_adf_read = vol_adf[vol_adf['cat'] == 'agg-read'].copy().reset_index()
vfd_adf_read = vfd_adf[vfd_adf['cat'] == 'agg-read'].copy().reset_index()

# get dset offsets from vol df
dset_offsets = list(set(vol_adf_read[vol_adf_read['func_name'] == 'H5VLdataset_read']['offset']))
# print(dset_offsets)

# # add data_label for vol
vol_adf_read['dset_name'] = vol_adf_read['dset_name'].fillna(method='bfill')
vol_adf_read['dset_name'] = vol_adf_read['dset_name'].fillna(method='ffill')
add_vol_op_type(vol_adf_read)
vol_data_label(vol_adf_read)


## add data_label for vfd
vfd_adf_read = vfd_op_data_label_agg_read(vfd_adf_read,dset_offsets)
# # df_to_csv(vfd_adf_read,fagg,suffix='-vfd-read')


df_to_csv(vol_adf_read,fagg,suffix='-vol-read')
df_to_csv(vfd_adf_read,fagg,suffix='-vfd-read')

## create lables for AGG_WRITE phase
1. create pattern for all 16B, 3KB I/O, and metadata and H5FD_MEM_LHEAP
2. create data_label for all 3KB I/O
3. map data_label with start_addr-3KB to end_addr-16B
** currently dataset is seperated with I/O size that is characteristic of point_cloud. Cannot work with other datasets for now.

In [7]:
from collections import Counter

def vfd_data_pattern_agg_write(df):
    op_type_list = []
    # tmp_row = pd.DataFrame(columns=list(df.columns))
    for index, row in df.iterrows():
        if row['mem_type'] == 'H5FD_MEM_DRAW':
            if row['access_size'] == 16:
                op_type_list.append('loc')
            else:
                op_type_list.append('data')
        elif row['mem_type'] == 'H5FD_MEM_LHEAP':
            op_type_list.append('lheap')
        else:
            op_type_list.append('meta')
    
    df['op_type'] = op_type_list
    return df

        
def vfd_data_label_agg_write(df, PC_IO_SIZE):
    # TODO: needs improvement

    data_idx = 0
    pc_idx = 0
    lheap_idx = 0
        
    data_label_list = []
    loc_map = {}
    data_label_map  = {}
    
    first_chunk_index = 0 # record index at dataframe
    prev_address = df[df['op_type'] == 'data'].iloc[0]['start_addr'] # first data address
    
    for index, row in df.iterrows():
        _pat = row['op_type']
        if _pat == 'data':
            if row['access_size'] == PC_IO_SIZE:
                data_label_list.append(f'vfd-data-point_cloud-{pc_idx}')
                pc_idx +=1
            else:
                _str = f'vfd-data-contact_map-{data_idx}'
                data_label_list.append(_str)
                if first_chunk_index == 0:
                    if row['start_addr'] != prev_address:
                        first_chunk_index = index
                else:
                    data_label_map[row['start_addr']] = _str
                data_idx +=1
                prev_address = row['next_addr']
                
        elif _pat == 'meta':
            data_label_list.append(f'vfd-{_pat}-1')
        elif _pat == 'lheap':
            data_label_list.append(f'vfd-{_pat}-1-{lheap_idx}')
            lheap_idx+=1
        else: 
            if loc_map:
                data_label_list.append('0')
            else:
                data_label_list.append(f'vfd-lheap-1')
            loc_map[row['next_addr']] = int(row['io_access_idx'])
    
    # map some 16B with data_label
    for k in data_label_map.keys():
        data_label_list[loc_map[k]] = data_label_map[k]
    
    # find and map the rest 16B with initial chunk
    indices = [i for i, x in enumerate(data_label_list) if x == '0']
    data_idx = 0
    for idx in indices:
        data_label_list[idx] = f'vfd-data-contact_map-{data_idx}'
        data_idx+=1

    df['data_label'] = data_label_list

def get_point_cloud_io_size(dfvol):
    idx = dfvol.dset_name.str.match('point_cloud').idxmax()
    # dims = ast.literal_eval(lastrow['dimensions'])
    first_dim = dfvol.iloc[idx]['dimensions'][0] #lastrow['dimensions'][idx]
    access_size = dfvol.iloc[idx]['access_size']
    io_size = access_size / first_dim
    return io_size

def hard_code_start_addr_agg(dfvol, dfvfd):
    # # hardcode starting address
    for dset in ['contact_map', 'point_cloud']:
        idx_vfd = dfvfd.data_label.str.match(f'vfd-data-{dset}-0').idxmax()
        START_ADDR = dfvfd.iloc[idx_vfd]['start_addr']
        
        idx_vol = dfvol.data_label.str.match(f'vol-data-{dset}-0').idxmax()
        dfvol.loc[idx_vol, ['logical_addr']] = START_ADDR

        idx_vol = dfvol.data_label.str.match(f'vol-data-{dset}-1').idxmax()
        if idx_vol:
            dfvol.loc[idx_vol, ['logical_addr']] = START_ADDR
    
    return dfvol

pattern_dict = { # for notes only now
    'mem_type' : {
        'H5FD_MEM_LHEAP' : 'lheap',
        'H5FD_MEM_DRAW' : { 'access_size':{ 16 : 'loc', 'others' : 'data'} },
        'H5FD_MEM_OHDR': 'meta',
        'H5FD_MEM_SUPER': 'meta'
    }
}

vfd_adf_write = vfd_adf[vfd_adf['cat'] == 'agg-write'].copy().reset_index()
vol_adf_write = vol_adf[vol_adf['cat'] == 'agg-write'].copy().reset_index()

# check_empty(vol_adf_write)

# add data_label for vol
vol_adf_write['dset_name'] = vol_adf_write['dset_name'].fillna(method='bfill')
vol_adf_write['dset_name'] = vol_adf_write['dset_name'].fillna(method='ffill')
add_vol_op_type(vol_adf_write)
vol_data_label(vol_adf_write)

vfd_adf_write = vfd_data_pattern_agg_write(vfd_adf_write)
PC_IO_SIZE = get_point_cloud_io_size(vol_adf_write)
vfd_data_label_agg_write(vfd_adf_write, PC_IO_SIZE)


# get logical address from vfd to vol
vol_adf_write = hard_code_start_addr_agg(vol_adf_write,vfd_adf_write)

df_to_csv(vol_adf_write,fagg,suffix='-vol-write')
df_to_csv(vfd_adf_write,fagg,suffix='-vfd-write')
# df_to_parquet(vol_adf_write,fagg,suffix='-vol-write')
# df_to_parquet(vfd_adf_write,fagg,suffix='-vfd-write')